In [1]:
!git clone https://github.com/TheMrGhostman/Semi-supervised-Learning-with-VAE.git

Cloning into 'Semi-supervised-Learning-with-VAE'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 560 (delta 80), reused 201 (delta 60), pack-reused 332
Receiving objects: 100% (560/560), 223.31 MiB | 36.40 MiB/s, done.
Resolving deltas: 100% (166/166), done.
Checking out files: 100% (371/371), done.


In [0]:
import os 
os.chdir("Semi-supervised-Learning-with-VAE/")

In [1]:
import numpy as np 
import time

import torch 
import torch.nn as nn
import torch.nn.functional as F 

import matplotlib.pyplot as plt
from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import RobustScaler

from utils.inference import Trainer, plot_loss
import utils.datasets as d

In [2]:
from utils.inception import Inception, InceptionBlock, correct_sizes
from utils import layers

In [3]:
X = np.vstack((np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt1.npy"),
               np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt2.npy")))
y = np.load("data/sequenced_data_for_VAE_length-160_stride-10_targets.npy")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)


In [5]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=666)

In [7]:
train = d.H_alphaSequences(X_train, y_train)
valid = d.H_alphaSequences(X_validation, y_validation)
test = d.H_alphaSequences(X_test, y_test)

In [11]:
class CRNN(nn.Module):
    def __init__(self, n_filters, kernel_sizes, rnn_out, sequence_len=160, bidirectional=True):
        super(CRNN, self).__init__()
        self.sequence_len = sequence_len
        # conv wants (batch, channel, length)
        self.reshape_to_inception = layers.Reshape(out_shape=(1, self.sequence_len))
        self.inception = Inception(
                in_channels=1, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                activation=nn.ReLU()
            )
        # RNN wants #(batch, seq, feature)
        self.rnn1 = nn.LSTM(
            input_size=n_filters*4,
            hidden_size=rnn_out*4,
            num_layers=1,
            batch_first=True,
            bidirectional=bidirectional
        )
        self.rnn2 = nn.LSTM(
            input_size=rnn_out*4*(2**bidirectional),
            hidden_size=rnn_out,
            num_layers=1,
            batch_first=True,
            bidirectional=bidirectional
        )
        #self.pool = nn.AdaptiveAvgPool1d(output_size=1)
        self.fc_on_rnn = nn.Linear(in_features=rnn_out*(2**bidirectional), out_features=1)
        self.flatten = layers.Flatten(out_features=self.sequence_len)
        self.fc = nn.Linear(in_features=self.sequence_len, out_features=4)

    def forward(self, x_in):
        x = self.reshape_to_inception(x_in)
        x = self.inception(x)
        #print(x.shape)
        x = x.permute(0,2,1)
        #print(x.shape)
        x, (h, c) = self.rnn1(x)
        x, (h, c) = self.rnn2(x)
        #print(x.shape)
        #x = self.pool(x)
        x = self.fc_on_rnn(x)
        #print(x.shape)
        x = self.flatten(x)
        #print(x.shape)
        return self.fc(x)

In [14]:
mod = CRNN(n_filters=32, kernel_sizes=[5,11,23], rnn_out=20)

In [13]:
mod(torch.tensor(X_train[0:2, :]).float())

tensor([[ 0.0713,  0.0648, -0.0135,  0.0477],
        [ 0.0684,  0.0446,  0.0148,  0.0494]], grad_fn=<AddmmBackward>)

In [15]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [16]:
optimizer= torch.optim.Adam(mod.parameters(), lr=1e-3)#5e-3

In [ ]:
m1 = Trainer(
        model=mod,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20,35], gamma=0.2),
        tensorboard=True,
        model_name="CRNN_lr-1e-3_bs-512_ks-[5,11,23]",
        verbose=True
        )

In [19]:
lh = m1(epochs=range(100), train_loader=train_loader, validation_loader=valid_loader)


Epoch [1/100], average_loss:0.8070, validation_loss:0.6234, val_accuracy:0.7403
Epoch [2/100], average_loss:0.5323, validation_loss:0.4393, val_accuracy:0.7922
Epoch [3/100], average_loss:0.4106, validation_loss:0.3750, val_accuracy:0.8571
Epoch [4/100], average_loss:0.3946, validation_loss:0.3893, val_accuracy:0.8182
Epoch [5/100], average_loss:0.3601, validation_loss:0.3765, val_accuracy:0.9091
Epoch [6/100], average_loss:0.3498, validation_loss:0.3099, val_accuracy:0.9091
Epoch [7/100], average_loss:0.3234, validation_loss:0.3666, val_accuracy:0.8831
Epoch [8/100], average_loss:0.3166, validation_loss:0.3001, val_accuracy:0.8701
Epoch [9/100], average_loss:0.3195, validation_loss:0.2886, val_accuracy:0.8831
Epoch [10/100], average_loss:0.3108, validation_loss:0.2842, val_accuracy:0.9481
Epoch [11/100], average_loss:0.2920, validation_loss:0.2776, val_accuracy:0.8701
Epoch [12/100], average_loss:0.2948, validation_loss:0.2635, val_accuracy:0.8831
Epoch [13/100], average_loss:0.2854, 

In [20]:
mod

CRNN(
  (reshape_to_inception): Reshape()
  (inception): Inception(
    (conv_from_bottleneck_1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (conv_from_bottleneck_2): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
    (conv_from_bottleneck_3): Conv1d(1, 32, kernel_size=(23,), stride=(1,), padding=(11,), bias=False)
    (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (conv_from_maxpool): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
    (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
  (rnn1): LSTM(128, 80, batch_first=True, bidirectional=True)
  (rnn2): LSTM(160, 20, batch_first=True, bidirectional=True)
  (fc_on_rnn): Linear(in_features=40, out_features=1, bias=True)
  (flatten): Flatten()
  (fc): Linear(in_features=160, out_features=4, bias=True)
)

In [22]:
m1.model.eval()
m1.model.cpu()
with torch.no_grad():
    x_pred = np.argmax(m1.model(torch.tensor(X_test).float()).detach(), axis=1)
m1.model.cuda()
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [23]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.9019984505524539

In [24]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9411448374760994

In [25]:
confusion_matrix(y_true=y_test, y_pred=x_pred) # x_axis = predicted, y_axis = ground_truth

array([[ 3876,   168,    36,   155],
       [  255, 10120,    13,    57],
       [   16,    10,   427,    26],
       [  187,    52,    10,  1328]], dtype=int64)

    InceptionTime Classification (best result so far)
    [ 3805,   296,    27,   107],
    [   91, 10307,     8,    39],
    [   22,     8,   425,    24],
    [  169,    37,    20,  1351]

In [26]:
torch.save(m1.model, "CRNN_lr-1e-3,_bs-512_ks-[5,11,23]_bidirectional.pt")
torch.save(m1.model.state_dict(), "CRNN_lr-1e-3,_bs-512_ks-[5,11,23]_bidirectional-state_dict.pt")

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Reshape. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Inception. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv1d. It won't be checked for correctness upon loa